In [ ]:
!pip install tweepy
!pip install pandas

In [18]:
from tqdm import tqdm
import pandas as pd
import tweepy
from pytz import timezone
from datetime import datetime
import pandas as pd

In [19]:
def get_api():
    consumer_key = "o5RkzZCWRv82BO3Q7Ou6g3gP6"
    consumer_secret = "ZkhVNVnl2AbiXcQNfFg3nyUujC8Im3dsaLWkTgtxY8bPMFWnFm"  
    access_token = "1511161028247138313-qJlKodPZHwX6CNHKhfkrVBRO57pj92"
    access_token_secret = "CTkfZ2vMesebR87k7i46T7qcSUqdOIv5oPmmjNS8XFxnX"

    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth,wait_on_rate_limit=True)
    
    return api

In [20]:
def get_data(keyword, number, day):
    api = get_api()
    tweets = tweepy.Cursor(api.search_tweets, q = keyword, tweet_mode = "extended", until=day).items(number)
    results = []
    for tweet in tqdm(tweets):
        results.append(tweet._json)
    return results

In [21]:
def exclude_spam(keyword,number,day):
    results = get_data(keyword,number,day)
    full_text = []
    for result in results:
        if 'retweeted_status' in result: # 중복제거 : retweet인 경우
            data = result['retweeted_status']
            if 'https://' in data['full_text']: # 스팸제거1 : https:// 포함시 필터
                continue
            else:
                if '\n' in data['full_text']: #스팸제거2 : 줄바꿈 문자 포함 중 조건에 맞지 않는 것 필터
                    if (len(data['full_text']) / data['full_text'].count('\n')) >= 10 and data['user']['followers_count'] > 20:
                        full_text.append({
                            'id_str':data['id_str'],
                            'screen_name':result['entities']['user_mentions'][0]['screen_name'],
                            'full_text':data['full_text'],
                            'retweeted':1
                        })
                    else:
                        continue
                else:
                    full_text.append({
                        'id_str':data['id_str'],
                        'screen_name':result['entities']['user_mentions'][0]['screen_name'],
                        'full_text':data['full_text'],
                        'retweeted':1
                    })
        else:
            if 'https://' in result['full_text']: #스팸제거1 : https:// 포함시 필터
                continue
            else:
                if '\n' in result['full_text']: #스팸제거2 : 줄바꿈 문자 포함 중 조건에 맞지 않는 것 필터
                    if (len(result['full_text']) / result['full_text'].count('\n')) >= 10 and result['user']['followers_count'] > 20:
                        full_text.append({
                            'id_str':result['user']['id_str'],
                            'screen_name':result['user']['screen_name'],
                            'full_text':result['full_text'],
                            'retweeted':0
                        })
                    else:
                        continue
                else:
                    full_text.append({
                            'id_str':result['user']['id_str'],
                            'screen_name':result['user']['screen_name'],
                            'full_text':result['full_text'],
                            'retweeted':0
                        })
    return full_text

In [25]:
from datetime import datetime, timedelta 
import time

def for_time(start,last,text):
    tweet_list = []
    count = 1
    # 시작일, 종료일 datetime 으로 변환 
    start_date = datetime.strptime(start, "%Y-%m-%d") 
    last_date = datetime.strptime(last, "%Y-%m-%d") 
    # 종료일 까지 반복 
    while start_date <= last_date:
        dates = start_date.strftime("%Y-%m-%d") 
        tweet_list.extend(exclude_spam(text,10000, dates))
        print('part',count,'clear')
        count += 1
        start_date += timedelta(days=1)
    return tweet_list

In [ ]:
tweet_korean = for_time('2022-05-13','2022-05-20','리그오브레전드')
df_korean = pd.DataFrame(tweet_korean, columns=['id_str', 'screen_name', 'full_text', 'retweeted'])
df_korean.to_csv('tweet_list.csv')

In [ ]:
tweet_top = for_time('2022-05-13','2022-05-20','롤 탑')
df_top = pd.DataFrame(tweet_top, columns=['id_str', 'screen_name', 'full_text', 'retweeted'])
df_top.to_csv('top_list.csv')

In [ ]:
tweet_mid = for_time('2022-05-13','2022-05-20','롤 미드')
df_mid = pd.DataFrame(tweet_mid, columns=['id_str', 'screen_name', 'full_text', 'retweeted'])
df_mid.to_csv('mid_list.csv')

In [40]:
tweet_jungle = for_time('2022-05-13','2022-05-20','롤 정글')
df_jungle = pd.DataFrame(tweet_jungle, columns=['id_str', 'screen_name', 'full_text', 'retweeted'])
df_jungle.to_csv('jungle_list.csv')

4it [00:01,  2.89it/s]


part 1 clear


17it [00:02,  8.10it/s]


part 2 clear


26it [00:02, 11.82it/s]


...

part 6 clear


157it [00:09, 17.32it/s]


part 7 clear


179it [00:09, 18.06it/s]

part 8 clear


In [41]:
tweet_ad = for_time('2022-05-13','2022-05-20','롤 원딜')
df_ad = pd.DataFrame(tweet_ad, columns=['id_str', 'screen_name', 'full_text', 'retweeted'])
df_ad.to_csv('ad_list.csv')

5it [00:01,  3.68it/s]


part 1 clear


9it [00:01,  6.44it/s]


part 2 clear


26it [00:02, 11.89it/s]


...

part 6 clear


57it [00:03, 15.08it/s]


part 7 clear


69it [00:04, 15.37it/s]

part 8 clear


In [42]:
tweet_sup = for_time('2022-05-13','2022-05-20','롤 서폿')
df_sup = pd.DataFrame(tweet_sup, columns=['id_str', 'screen_name', 'full_text', 'retweeted'])
df_sup.to_csv('sup_list.csv')

8it [00:01,  5.78it/s]


part 1 clear


11it [00:01,  7.85it/s]


part 2 clear


27it [00:02, 12.54it/s]


...

part 6 clear


67it [00:04, 14.95it/s]


part 7 clear


77it [00:05, 15.07it/s]

part 8 clear
